In [2]:
#Langchain
from langchain.tools import Tool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.agents import initialize_agent, AgentType

In [3]:
import warnings
warnings.filterwarnings('ignore')

### make a .env file and save the openai api key as shown below 
OPENAI_API_KEY="sk-proj-M4fGFE8f_VC"

In [4]:
# Working with ChatOpenAI from LangChain
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# Initialize the OpenAI Chat model
chat_model = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=api_key)

# Send a message to the model and get the response
response = chat_model.invoke("what is the capital of India?")
print(response.content)

The capital of India is New Delhi.


In [5]:
# 1 prompt templates
from langchain.prompts import PromptTemplate

# Define a simple prompt template
prompt_template = PromptTemplate(
    input_variables=["name"],
    template="Hello, {name}! How can I help you today?"
)

# Generate the prompt by filling in the variable
formatted_prompt = prompt_template.format(name="Neural network")
print(formatted_prompt) 

Hello, Neural network! How can I help you today?


In [6]:
# now we can use the prompt template with the chat model
#LLM Chains

llm = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=api_key)

learn_template = """
I want you to act as a consultant for a AI training
Return a list of topics and why it is important to learn in given area of AI
The description should be relevant to recent advancement in AI
What are some good topics to learn in {AI_topic}
"""

prompt_template = PromptTemplate(
    input_variables=["AI_topic"],
    template=learn_template,
)

description = "Please suggest me topics to learn in Deep learning"

prompt_template.format(AI_topic=description)

chain = LLMChain(llm=llm, prompt=prompt_template)

print(chain.run(description))

1. Neural Networks: Neural networks are the building blocks of deep learning and have been instrumental in the recent advancements in AI. Understanding how neural networks work and how to optimize them is crucial for developing effective deep learning models.

2. Convolutional Neural Networks (CNNs): CNNs have revolutionized image recognition and computer vision tasks. Learning how CNNs work and how to design and train them effectively is essential for anyone interested in working with image data.

3. Recurrent Neural Networks (RNNs) and Long Short-Term Memory (LSTM) networks: RNNs and LSTMs are essential for working with sequential data such as time series data, text data, and speech data. Understanding how to use these types of networks can enable you to build powerful predictive models.

4. Generative Adversarial Networks (GANs): GANs have gained a lot of attention in recent years for their ability to generate realistic images, videos, and text. Learning how GANs work and how to tra

In [9]:
from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool

# Define a simple tool
def my_tool_function(query: str) -> str:
    return f"Tool response: {query}"

#creating tool from function
my_tool = Tool.from_function(func=my_tool_function, name="simple_tool", description="A simple tool")

ddg_search = DuckDuckGoSearchRun()

# Initialize the LLM and the agent
llm = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=api_key)
tools = [my_tool,ddg_search]

# Create an agent
agent = initialize_agent(tools, llm, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Use the agent
response = agent.run("What's the weather like today in Bengaluru?")
print(response)  # Output: The agent will choose to use the tool or model to respond based on context



> Entering new AgentExecutor chain...
I should use a tool to check the weather forecast for Bengaluru.
Action: simple_tool
Action Input: "weather in Bengaluru today"
Observation: Tool response: weather in Bengaluru today
Thought:I should verify this information by searching on DuckDuckGo.
Action: duckduckgo_search
Action Input: "weather forecast Bengaluru today"
Observation: Bengaluru begins the week today, June 16, with a cool, damp morning featuring patchy rain and temperatures between 19.6°C and 23.1°C. ... The weather forecast for the upcoming week shows a ... The Long-range 12 day forecast also includes detail for Bengaluru weather today. Live weather reports from Bengaluru weather stations and weather warnings that include risk of thunder, high UV index and forecast gales. See the links below the 12-day Bengaluru weather forecast table for other cities and towns nearby along with weather conditions ... The weather today in Bangalore will be hot with temperatures reaching 27 ° C

In [11]:
# Define a tool to get the current date and time
import datetime

def get_current_datetime(timezone=None):
    """Get the current date and time, optionally for a specific timezone"""
    if timezone:
        # In a real implementation, we would handle different timezones
        return f"Current date and time for {timezone}: {datetime.datetime.now()}"
    else:
        return f"Current date and time: {datetime.datetime.now()}"

# Create a tool from the function
datetime_tool = Tool.from_function(
    func=get_current_datetime,
    name="DateTimeTool",
    description="Useful for getting the current date and time, optionally for a specific timezone"
)

In [10]:
# Create a summarization prompt template
summarize_template = PromptTemplate(
    input_variables=["content"],
    template="Summarize the following content in 3-5 key points: {content}"
)

# Create a summarization chain
summarize_chain = LLMChain(
    llm=llm,
    prompt=summarize_template
)

# Create a summarization tool
summarize_tool = Tool.from_function(
    func=lambda text: summarize_chain.run({"content": text}),
    name="Summarizer",
    description="Useful for summarizing long text content into key points"
)

In [12]:
# Define our tools
tools = [datetime_tool, summarize_tool,ddg_search] 

# Initialize the agent with search and summarization tools
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [13]:
response = agent.run("What are the best places to visit in Japan for first-time visitors?")
print(f"Japan Travel Response: {response}")



> Entering new AgentExecutor chain...
I should use DuckDuckGo search to find a list of popular tourist destinations in Japan 
Action: duckduckgo_search
Action Input: "best places to visit in Japan for first-time visitors"
Observation: This Japan itinerary starts in Tokyo, makes a day trip to either Kamakura, Nikko, or Hakone, then ventures south to Kyoto, with day trips to Nara, Osaka, Hiroshima, and Miyajima Island.Distances really depend on the mode of transport you use, with bullet trains being the fastest. Days 1-3: Tokyo; Day 4: day trip from Tokyo Days 5-6: Kyoto Day 7: Nara and Osaka *This is a daily estimate only. The photo on the right shows my actual travel expenses from my recent Japan trip during Taylor Swift's Eras Tour. My budget for the entire 11-day Japan itinerary, starting in Osaka and ending in Tokyo, was 120,000 pesos, but I ended up spending 113,712 pesos (USD 2,000 or JPY 311,000). Although exchange rates have fluctuated over the past few months, this ... TOP 5 